In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Define the CNN model
class CNN_Model(nn.Module):
    def __init__(self):
        super(CNN_Model, self).__init__()
        
        # First convolutional layer: input 1x28x28, output 32x28x28
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        # Second convolutional layer: input 32x28x28, output 64x28x28
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        
        # Max pooling layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        
        # Fully connected layers
        self.fc1 = nn.Linear(64 * 7 * 7, 128)  # Flattened 7x7x64
        self.fc2 = nn.Linear(128, 10)  # Output layer for 10 classes

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))  # Apply Conv1 and Pooling
        x = self.pool(torch.relu(self.conv2(x)))  # Apply Conv2 and Pooling
        x = x.view(-1, 64 * 7 * 7)  # Flatten the image for the fully connected layers
        x = torch.relu(self.fc1(x))  # Fully connected layer 1
        x = self.fc2(x)  # Output layer
        return x

# Load MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=64, shuffle=False)

# Initialize the model, loss function, and optimizer
model = CNN_Model()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training the model
num_epochs = 5
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(trainloader, 0):
        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss / len(trainloader)}")

# Evaluate the model on the test set
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in testloader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on test set: {100 * correct / total}%')


Epoch 1, Loss: 0.14053734809856241


In [1]:
import torch
import torch.nn as nn

# Define the DoubleInputLSTM model
class DoubleInputLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(DoubleInputLSTM, self).__init__()
        
        # Define two separate LSTM layers for the two inputs
        self.lstm1 = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.lstm2 = nn.LSTM(input_size, hidden_size, batch_first=True)
        
        # Fully connected layer to combine the two LSTM outputs and produce final output
        self.fc = nn.Linear(hidden_size * 2, output_size)
    
    def forward(self, input1, input2):
        # Pass the first input through the first LSTM
        lstm1_out, (h1, c1) = self.lstm1(input1)
        
        # Pass the second input through the second LSTM
        lstm2_out, (h2, c2) = self.lstm2(input2)
        
        # Concatenate the LSTM outputs of both inputs (we concatenate at the feature dimension)
        combined_out = torch.cat((lstm1_out, lstm2_out), dim=-1)
        
        # Pass the combined output through a fully connected layer
        output = self.fc(combined_out)
        
        return output


# Example usage of the model
batch_size = 32
input_size = 2464
hidden_size = 200
output_size = 2464  # Same as input_size

# Create random input data for two sequences (e.g., two different feature sequences)
input1 = torch.randn(batch_size,  input_size)
input2 = torch.randn(batch_size,  input_size)

# Instantiate the model
model = DoubleInputLSTM(input_size, hidden_size, output_size)

# Forward pass through the model
output = model(input1, input2)

# Output shape should be (batch_size, seq_length, output_size)
print(f"Output shape: {output.shape}")


Output shape: torch.Size([32, 2464])


In [ ]:
#https://shap.readthedocs.io/en/latest/example_notebooks/text_examples/question_answering/Explaining%20a%20Question%20Answering%20Transformers%20Model.html
#https://shap.readthedocs.io/en/latest/example_notebooks/image_examples/image_classification/Explain%20MobilenetV2%20using%20the%20Partition%20explainer%20%28PyTorch%29.html